In [1]:
import mpmath as mpmath
import mpmath as mp
mp.mp.dps = 30 #higher precision
import pandas as pd
import os

In [2]:
import re
import json
from collections import defaultdict
import mpmath as mp
import scipy

mp.mp.dps = 30  # higher precision


def log_sum_exp(to_sum):
    maxval = max(to_sum)
    exp_sum = 0
    for value in to_sum:
        exp_sum += mp.exp(value - maxval)
    res = maxval + mp.log(exp_sum)
    return res


# Free energy given histogram and temperature, arbitrary precision governed by mp
def free_energy(E_list, log_g_list, T):
    to_sum = []
    for i, log_g in enumerate(log_g_list):
        to_sum.append(log_g - E_list[i] / T)
    maxval = max(to_sum)
    exp_sum = 0
    for value in to_sum:
        exp_sum += mp.exp(value - maxval)
    res = maxval + mp.log(exp_sum)
    return -T * res


# Run over batch of results, structured by seed, then by class
def get_free_energies(rescaled_results, temperatures):
    free_energies = []
    for seed_results in rescaled_results:
        free_energy_classes = []
        for error_result in seed_results:
            f_values = []
            for T in temperatures:
                f_values.append(free_energy(error_result[0], error_result[1], T) / (-T))
            free_energy_classes.append(f_values)
        free_energies.append(free_energy_classes)
    return free_energies


def read_results_file(path):

    with open(path, "r") as file:
        content = file.read()

    content = content.strip().rstrip(",")

    corrected_json = f"[{content}]"

    try:
        data = json.loads(corrected_json)
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON: {e}")

    return data


def process_data(data, batch_results, p, X, Y, error):
    for entry in data:
        histogram_seed = entry["histogram_seed"]
        run_seed = entry["run_seed"]
        results = entry["results"]

        E_list = []
        log_g_list = []

        # Process the results
        for key, value in results.items():
            E_list.append(int(key))
            log_g_list.append(float(value))

        batch_results.append(
            {
                "prob": p,
                "X": X,
                "Y": Y,
                "error": error,
                "histogram_seed": histogram_seed,
                "run_seed": run_seed,
                "E": E_list,
                "log_g": log_g_list,
            }
        )


def filter_and_normalize_rbim(batch_results):
    # grouped dictionary with keys prob size and hist seed
    grouped_results = defaultdict(list)
    for result in batch_results:
        key = (result["prob"], result["X"], result["Y"], result["histogram_seed"])
        grouped_results[key].append(result)

    filtered_results = defaultdict(list)
    for key, results in grouped_results.items():
        newkey = (key[0], key[1], key[1])
        errors = set(result["error"] for result in results)
        if errors == {"I", "X", "Y", "Z"}:
            # To be removed once normalization is properly handled in c
            for result in results:
                log_g_list = result["log_g"]
                offset = log_sum_exp(log_g_list)
                rescaled_log_g_list = [
                    res + mp.log(2) * key[1] * key[2] - offset for res in log_g_list
                ]
                result["log_g"] = rescaled_log_g_list
            filtered_results[newkey].append(
                [[result["E"], result["log_g"]] for result in results]
            )
        else:
            print(
                f"has issue with an error class prob: {key[0]} X: {key[1]} Y: {key[2]} interaction seed: {key[2]} available errors: {errors}"
            )

    return filtered_results


def calc_free_energies_batch(filtered_results, probability, X, Y):
    T_Nish = 1 / (mp.log((1 - probability) / probability) / 2)
    temperatures = [1e-20, T_Nish, 1e20]
    batch_res = filtered_results[(probability, X, Y)]
    free_energies = get_free_energies(batch_res, temperatures)
    print(
        "Number of seeds at p", probability, ", X", X, ", Y", Y, ":", len(free_energies)
    )
    return free_energies


def calculate_curve(free_energies, temp):
    """Calculate the curve value for a given temperature."""
    return 1 - mp.fsum(
        [
            f_class[0][temp] < f_class[1][temp]
            or f_class[0][temp] < f_class[2][temp]
            or f_class[0][temp] < f_class[3][temp]
            for f_class in free_energies
        ]
    ) / len(free_energies)


def calculate_bounds(successes, failures, curve):
    """Calculate the confidence interval bounds."""
    lower_bound = curve - scipy.stats.beta.ppf(0.025, 0.5 + successes, 0.5 + failures)
    upper_bound = scipy.stats.beta.ppf(0.025, 0.5 + successes, 0.5 + failures) - curve
    return lower_bound, upper_bound


def get_optimal_curves(free_energies):
    """Calculate optimal and T0 curves and their confidence intervals."""

    # Calculate curves for two different temperatures
    optimal_curve = calculate_curve(free_energies, temp=1)  # Nishimori temp
    T0_curve = calculate_curve(free_energies, temp=0)  # Low temp

    # Number of successes and failures for both curves
    total_len = len(free_energies)
    number_success_optimal = round(total_len * optimal_curve)
    number_failure_optimal = total_len - number_success_optimal

    number_success_T0 = round(total_len * T0_curve)
    number_failure_T0 = total_len - number_success_T0

    # Calculate confidence interval bounds
    lower_bounds_T0, upper_bounds_T0 = calculate_bounds(
        number_success_T0, number_failure_T0, T0_curve
    )
    lower_bounds_optimal, upper_bounds_optimal = calculate_bounds(
        number_success_optimal, number_failure_optimal, optimal_curve
    )

    return {
        "optimal_curve": optimal_curve,
        "T0_curve": T0_curve,
        "lower_bounds_T0": lower_bounds_T0,
        "upper_bounds_T0": upper_bounds_T0,
        "lower_bounds_optimal": lower_bounds_optimal,
        "upper_bounds_optimal": upper_bounds_optimal,
    }


In [3]:
def rescale_log_g(row):
    log_g_list = row["log_g"]
    X = row["X"]
    Y = row["Y"]

    offset = log_sum_exp(log_g_list)

    rescaled_log_g_list = [
        res + mp.log(2)*X*Y - offset for res in log_g_list
    ]

    return rescaled_log_g_list

In [4]:
boundary_type = "periodic"
intervals = 16
iterations = 1000
overlap = 0.25
walkers = 4
alpha = 0.8
beta = 1e-8
exchange = 10

prob_x = 0.12
prob_y = 0.000100
prob_z = 0.000100
X = 4
Y = 4
errors = ["0000", "0100", "1000", "1100"]

batch_results = []

for error in errors:
    filename = f"../../../results/eight_vertex/bit_flip_limit/prob_x_{prob_x:.6f}/prob_y_{prob_y:.6f}/prob_z_{prob_z:.6f}/X_{X}_Y_{Y}/error_class_{error}/StitchedHistogram__intervals_{intervals}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset_{exchange}.txt"

    data = read_results_file(filename)
    if data:
        process_data(data, batch_results, prob_x, X, Y, error)

df = pd.DataFrame(batch_results)


In [5]:
df

,prob,X,Y,error,histogram_seed,run_seed,E,log_g
0,0.12,4,4,0000,5446,9,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.06439346075058, 3.663536578416824, 4.9..."
1,0.12,4,4,0000,3054,9,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.045654281973839, 3.680158644914627, 4...."
2,0.12,4,4,0000,5547,9,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 1.923056125640869, 3.440548539161682, 4...."
3,0.12,4,4,0000,3253,9,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.059100061655045, 3.71755825728178, 5.0..."
4,0.12,4,4,0000,6780,9,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.214848764240742, 3.91125850379467, 5.1..."
...,...,...,...,...,...,...,...,...
19969,0.12,4,4,1100,5565,12,"[-22, -18, -14, -10, -6, -2, 2, 6, 10, 14, 18,...","[0.0, 4.352355189621449, 5.975252620875835, 7...."
19970,0.12,4,4,1100,6829,12,"[-18, -14, -10, -6, -2, 2, 6, 10, 14, 18]","[0.0, 3.457518488168716, 5.166221715509892, 6...."
19971,0.12,4,4,1100,3898,12,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 4.797202050685883, 7.026054069399834, 8...."
19972,0.12,4,4,1100,4107,12,"[-18, -14, -10, -6, -2, 2, 6, 10, 14, 18]","[0.0, 5.111525982618332, 7.136123709380627, 8...."


In [6]:

df['error'] = df['error'].astype(str)
# Filter based on errors
df_filtered = df.groupby('histogram_seed').filter(
    lambda x: len(x) == len(errors)
)

df_filtered['rescaled_log_g'] = df_filtered.apply(rescale_log_g, axis=1)

df_sorted = df_filtered.sort_values(by=["histogram_seed", "error"])

nish_temp = 1/(mp.log((1-prob_x)/prob_x)/2)

df_sorted[f"free_energy_nish_temp"] = df_sorted.apply(lambda row: free_energy(row['E'], row['rescaled_log_g'], nish_temp)/(-nish_temp), axis=1)
print(nish_temp)


1.00379929768377641274902651412


In [7]:
df_sorted.sort_values(by='histogram_seed')

,prob,X,Y,error,histogram_seed,run_seed,E,log_g,rescaled_log_g,free_energy_nish_temp
455,0.12,4,4,0000,3000,9,"[-28, -24, -20, -16, -12, -8, -4, 0, 4, 8, 12,...","[0.0, 1.38192730024457, 3.410100761801004, 5.1...","[0.716128978484095887829902891, 2.098056278728...",28.6924101884075054520598620196
5514,0.12,4,4,0100,3000,10,"[-28, -24, -20, -16, -12, -8, -4, 0, 4, 8, 12,...","[0.0, 1.399247951805592, 3.469382483512163, 5....","[0.582321448827120793844302616646, 1.981569400...",28.5605569930607664591490587728
10550,0.12,4,4,1000,3000,11,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.536376781761646, 4.067395552992821, 5....","[3.44453449244138665186630890273, 5.9809112742...",23.5971866092644111024386381207
15778,0.12,4,4,1100,3000,12,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.549784511327744, 4.144676350057125, 5....","[3.47679631393749511595945667307, 6.0265808252...",23.6332738906902156797503954597
4232,0.12,4,4,0000,3001,9,"[-26, -22, -18, -14, -10, -6, -2, 2, 6, 10, 14...","[0.0, 2.480150803923607, 4.319334845989943, 6....","[0.653660804158001116721486686508, 3.133811608...",26.7793424597570228095345316177
...,...,...,...,...,...,...,...,...,...,...
7478,0.12,4,4,0100,7998,10,"[-22, -18, -14, -10, -6, -2, 2, 6, 10, 14, 18,...","[0.0, 1.868415243923664, 4.106914199888706, 5....","[2.59219565207838570897829520695, 4.4606108960...",24.6430204508948154411014930268
9236,0.12,4,4,0100,7999,10,"[-24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, ...","[0.0, 2.984251823276281, 5.075812187045813, 6....","[0.565565086932785637410680954984, 3.549816910...",24.8325359743365230892116110253
10627,0.12,4,4,1000,7999,11,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.348172724246979, 3.941430859267712, 5....","[3.745513062874570070767141533, 6.093685787121...",23.8633313308010670734804319588
2705,0.12,4,4,0000,7999,9,"[-24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, ...","[0.0, 2.107758935540915, 3.92784334346652, 5.4...","[2.10597502460843274256338219684, 4.2137339601...",26.1740149820851805019430132558


In [8]:
import re

def read_results_file_without_comma_seperation(path):
    with open(path, 'r') as file:
        content = file.read()

    # Add a comma between each dictionary block
    content = content.strip()
    content = content.replace('}\n{', '},\n{')

    # Wrap the content in square brackets to make it a valid JSON array
    corrected_json = f'[{content}]'

    try:
        data = json.loads(corrected_json)
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON: {e}")
        return None

    return data

def read_results_file_pure_bitflip(path):

    with open(path, 'r') as file:
        content = file.read()

    content = content.strip().rstrip(',')

    corrected_json = f'[{content}]'

    try:
        data = json.loads(corrected_json)
    except json.JSONDecodeError as e:
        raise RuntimeError(f"Failed to parse JSON: {e}")


    return data

def parse_file(filename):
    data = []
    try:
        with open(filename, 'r') as file:
            content = file.read()
            # print("file content printout (for debugging):")
            # print(content)
    except FileNotFoundError:
        return

    # Split content into individual blocks
    blocks = content.split('}\n{')  # Assuming blocks are separated by double newlines

    for block in blocks:
        # Extract histogram_seed, run_seed, and results
        histogram_seed_match = re.search(r'"histogram_seed": "(\d+)"', block)
        run_seed_match = re.search(r'"run_seed": "(\d+)"', block)
        results_match = re.search(r'"results": \[([^]]*)\]', block)

        if histogram_seed_match and run_seed_match and results_match:
            histogram_seed = histogram_seed_match.group(1)
            run_seed = run_seed_match.group(1)
            results_str = results_match.group(1)

            # Process results
            results = {}
            results_items = results_str.split(',')
            for item in results_items:
                key_value = item.split(':')
                if len(key_value) == 2:
                    key = key_value[0].strip().strip('"')
                    value = float(key_value[1].strip())
                    results[key] = value

            data.append({
                "histogram_seed": histogram_seed,
                "run_seed": run_seed,
                "results": results
            })

    return data

def process_data(data, batch_results, p, size, error):
    for entry in data:
        histogram_seed = entry["histogram_seed"]
        run_seed = entry["run_seed"]
        results = entry["results"]

        E_list = []
        log_g_list = []

        # Process the results
        for key, value in results.items():
            E_list.append(int(key))
            log_g_list.append(float(value))

        batch_results.append({
                'prob': p,
                'size': size,
                'error': error,
                'histogram_seed': histogram_seed,
                'run_seed': run_seed,
                'E': E_list,
                'log_g': log_g_list
            })
        # offset = log_sum_exp(log_g_list)
        # print('deviation from sqrt(2)xy: ', mp.log(2)*size*size - offset, ' log sum exp:', offset)

def rescale_log_g(log_g_list, size):
    offset = log_sum_exp(log_g_list)
    rescaled_log_g_list = [res + mp.log(2) * size * size - offset for res in log_g_list]
    return rescaled_log_g_list


def check_condition(group):
    if len(group) != 4:
        return pd.Series({'There are still error classes missing': None})
    f_0000 = group.loc[group['error'] == '0000', 'free_energy'].values[0]

    f_others = group.loc[group['error'].isin(['0100', '1000', '1100']), 'free_energy']

    return pd.Series({'success': int(f_0000 < f_others.min())})

In [9]:

# Eight Vertex limit read
batch_results = []
probabilities = [0.12, 0.13]
sizes = [4]
intervals = [16]
iterations = 1000
overlap = 0.25
walkers = 4
alpha = 0.8
beta = 1e-8
exchange = 10

batch_results = []

for interval in intervals:
    for p in probabilities:
        for size in sizes:
            for error in ["0000", "0100", "1000", "1100"]:
                filename = f"../../../results/eight_vertex/bit_flip_limit/prob_x_{p:.6f}/prob_y_0.000100/prob_z_0.000100/X_{size}_Y_{size}/error_class_{error}/StitchedHistogram__intervals_{interval}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset_{exchange}.txt"
                if os.path.exists(filename):
                    try:
                        data = read_results_file_pure_bitflip(filename)
                        if data:
                            process_data(data, batch_results, p, size, error)
                    except:
                        data = read_results_file_without_comma_seperation(filename)
                        if data:
                            process_data(data, batch_results, p, size, error)
print("eight vertex limit results: ", len(batch_results))


eight vertex limit results:  39947


In [10]:
# Get free energies
batch_with_free_energy = []


p=0.12
T_Nish = 1/(mp.log((1-p)/p)/2)
T = T_Nish

batch_df = pd.DataFrame(batch_results)

batch_df['rescaled_log_g'] = batch_df.apply(lambda row: rescale_log_g(row['log_g'], row['size']), axis=1)
print(T)
batch_df['free_energy'] = batch_df.apply(lambda row: free_energy(row['E'], row['rescaled_log_g'], T)/(-T), axis=1)


1.00379929768377641274902651412


In [11]:
batch_df.sort_values(by='histogram_seed')

,prob,size,error,histogram_seed,run_seed,E,log_g,rescaled_log_g,free_energy
455,0.12,4,0000,3000,9,"[-28, -24, -20, -16, -12, -8, -4, 0, 4, 8, 12,...","[0.0, 1.38192730024457, 3.410100761801004, 5.1...","[0.716128978484095887829902891, 2.098056278728...",28.6924101884075054520598620196
5514,0.12,4,0100,3000,10,"[-28, -24, -20, -16, -12, -8, -4, 0, 4, 8, 12,...","[0.0, 1.399247951805592, 3.469382483512163, 5....","[0.582321448827120793844302616646, 1.981569400...",28.5605569930607664591490587728
30230,0.13,4,1000,3000,19,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.339791506528854, 3.923378400504589, 5....","[3.64655112306633418757834304447, 5.9863426295...",23.7628473475768223668148546363
10550,0.12,4,1000,3000,11,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.536376781761646, 4.067395552992821, 5....","[3.44453449244138665186630890273, 5.9809112742...",23.5971866092644111024386381207
20609,0.13,4,0000,3000,17,"[-28, -24, -20, -16, -12, -8, -4, 0, 4, 8, 12,...","[0.0, 1.400073532015085, 3.393615495413542, 5....","[0.694261970153086027891402788058, 2.094335502...",28.6716606551460556421530880348
...,...,...,...,...,...,...,...,...,...
2705,0.12,4,0000,7999,9,"[-24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, ...","[0.0, 2.107758935540915, 3.92784334346652, 5.4...","[2.10597502460843274256338219684, 4.2137339601...",26.1740149820851805019430132558
9236,0.12,4,0100,7999,10,"[-24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, ...","[0.0, 2.984251823276281, 5.075812187045813, 6....","[0.565565086932785637410680954984, 3.549816910...",24.8325359743365230892116110253
28822,0.13,4,0100,7999,18,"[-24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, ...","[0.0, 3.002470564097166, 4.985491551458836, 6....","[0.60720495431294238524286576534, 3.6096755184...",24.8753083737803012147117181823
30568,0.13,4,1000,7999,19,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.238416120409966, 3.956438146531582, 5....","[3.66867706480592897483152434215, 5.9070931852...",23.7698416875794132734527703465


In [12]:
df_sorted

,prob,X,Y,error,histogram_seed,run_seed,E,log_g,rescaled_log_g,free_energy_nish_temp
455,0.12,4,4,0000,3000,9,"[-28, -24, -20, -16, -12, -8, -4, 0, 4, 8, 12,...","[0.0, 1.38192730024457, 3.410100761801004, 5.1...","[0.716128978484095887829902891, 2.098056278728...",28.6924101884075054520598620196
5514,0.12,4,4,0100,3000,10,"[-28, -24, -20, -16, -12, -8, -4, 0, 4, 8, 12,...","[0.0, 1.399247951805592, 3.469382483512163, 5....","[0.582321448827120793844302616646, 1.981569400...",28.5605569930607664591490587728
10550,0.12,4,4,1000,3000,11,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.536376781761646, 4.067395552992821, 5....","[3.44453449244138665186630890273, 5.9809112742...",23.5971866092644111024386381207
15778,0.12,4,4,1100,3000,12,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.549784511327744, 4.144676350057125, 5....","[3.47679631393749511595945667307, 6.0265808252...",23.6332738906902156797503954597
4232,0.12,4,4,0000,3001,9,"[-26, -22, -18, -14, -10, -6, -2, 2, 6, 10, 14...","[0.0, 2.480150803923607, 4.319334845989943, 6....","[0.653660804158001116721486686508, 3.133811608...",26.7793424597570228095345316177
...,...,...,...,...,...,...,...,...,...,...
18278,0.12,4,4,1100,7998,12,"[-22, -18, -14, -10, -6, -2, 2, 6, 10, 14, 18,...","[0.0, 2.997990932315588, 4.983085673302412, 6....","[1.63835070709506881263636198304, 4.6363416394...",23.9115629566518239253600674677
2705,0.12,4,4,0000,7999,9,"[-24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, ...","[0.0, 2.107758935540915, 3.92784334346652, 5.4...","[2.10597502460843274256338219684, 4.2137339601...",26.1740149820851805019430132558
9236,0.12,4,4,0100,7999,10,"[-24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, ...","[0.0, 2.984251823276281, 5.075812187045813, 6....","[0.565565086932785637410680954984, 3.549816910...",24.8325359743365230892116110253
10627,0.12,4,4,1000,7999,11,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.348172724246979, 3.941430859267712, 5....","[3.745513062874570070767141533, 6.093685787121...",23.8633313308010670734804319588


In [13]:
joined_df = pd.merge(df_sorted, batch_df, left_on=['prob', 'X', 'error', 'histogram_seed', 'run_seed'], right_on=['prob', 'size', 'error', 'histogram_seed', 'run_seed'], how='inner')

In [14]:
joined_df['rescaled_log_g_diff'] = joined_df.apply(lambda row: [a - b for a, b in zip(row['rescaled_log_g_x'], row['rescaled_log_g_y'])], axis=1)

In [15]:
joined_df

,prob,X,Y,error,histogram_seed,run_seed,E_x,log_g_x,rescaled_log_g_x,free_energy_nish_temp,size,E_y,log_g_y,rescaled_log_g_y,free_energy,rescaled_log_g_diff
0,0.12,4,4,0000,3000,9,"[-28, -24, -20, -16, -12, -8, -4, 0, 4, 8, 12,...","[0.0, 1.38192730024457, 3.410100761801004, 5.1...","[0.716128978484095887829902891, 2.098056278728...",28.6924101884075054520598620196,4,"[-28, -24, -20, -16, -12, -8, -4, 0, 4, 8, 12,...","[0.0, 1.38192730024457, 3.410100761801004, 5.1...","[0.716128978484095887829902891, 2.098056278728...",28.6924101884075054520598620196,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0.12,4,4,0100,3000,10,"[-28, -24, -20, -16, -12, -8, -4, 0, 4, 8, 12,...","[0.0, 1.399247951805592, 3.469382483512163, 5....","[0.582321448827120793844302616646, 1.981569400...",28.5605569930607664591490587728,4,"[-28, -24, -20, -16, -12, -8, -4, 0, 4, 8, 12,...","[0.0, 1.399247951805592, 3.469382483512163, 5....","[0.582321448827120793844302616646, 1.981569400...",28.5605569930607664591490587728,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0.12,4,4,1000,3000,11,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.536376781761646, 4.067395552992821, 5....","[3.44453449244138665186630890273, 5.9809112742...",23.5971866092644111024386381207,4,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.536376781761646, 4.067395552992821, 5....","[3.44453449244138665186630890273, 5.9809112742...",23.5971866092644111024386381207,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0.12,4,4,1100,3000,12,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.549784511327744, 4.144676350057125, 5....","[3.47679631393749511595945667307, 6.0265808252...",23.6332738906902156797503954597,4,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.549784511327744, 4.144676350057125, 5....","[3.47679631393749511595945667307, 6.0265808252...",23.6332738906902156797503954597,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0.12,4,4,0000,3001,9,"[-26, -22, -18, -14, -10, -6, -2, 2, 6, 10, 14...","[0.0, 2.480150803923607, 4.319334845989943, 6....","[0.653660804158001116721486686508, 3.133811608...",26.7793424597570228095345316177,4,"[-26, -22, -18, -14, -10, -6, -2, 2, 6, 10, 14...","[0.0, 2.480150803923607, 4.319334845989943, 6....","[0.653660804158001116721486686508, 3.133811608...",26.7793424597570228095345316177,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19891,0.12,4,4,1100,7998,12,"[-22, -18, -14, -10, -6, -2, 2, 6, 10, 14, 18,...","[0.0, 2.997990932315588, 4.983085673302412, 6....","[1.63835070709506881263636198304, 4.6363416394...",23.9115629566518239253600674677,4,"[-22, -18, -14, -10, -6, -2, 2, 6, 10, 14, 18,...","[0.0, 2.997990932315588, 4.983085673302412, 6....","[1.63835070709506881263636198304, 4.6363416394...",23.9115629566518239253600674677,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19892,0.12,4,4,0000,7999,9,"[-24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, ...","[0.0, 2.107758935540915, 3.92784334346652, 5.4...","[2.10597502460843274256338219684, 4.2137339601...",26.1740149820851805019430132558,4,"[-24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, ...","[0.0, 2.107758935540915, 3.92784334346652, 5.4...","[2.10597502460843274256338219684, 4.2137339601...",26.1740149820851805019430132558,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19893,0.12,4,4,0100,7999,10,"[-24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, ...","[0.0, 2.984251823276281, 5.075812187045813, 6....","[0.565565086932785637410680954984, 3.549816910...",24.8325359743365230892116110253,4,"[-24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, ...","[0.0, 2.984251823276281, 5.075812187045813, 6....","[0.565565086932785637410680954984, 3.549816910...",24.8325359743365230892116110253,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19894,0.12,4,4,1000,7999,11,"[-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20]","[0.0, 2.348172724246979, 3.941430859267712, 5....","[3.74551306287457007076714

In [16]:
import numpy as np
# ['rescaled_log_g_diff'] = joined_df['rescaled_log_g_diff'].apply(lambda row: np.max(row['rescaled_log_g_diff']))
joined_df['Max_A'] = joined_df['rescaled_log_g_diff'].apply(max)

In [17]:
display(joined_df['Max_A'].unique())

array([mpf('0.0')], dtype=object)